In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [9]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch < 15:
        return 0.001
    elif epoch < 30:
        return 0.0005
    elif epoch < 45:
        return 0.0001
    else:
        return 0.00005

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        early_stopping = EarlyStopping(monitor = 'val_loss', patience = 20, verbose = 0, mode= 'min')
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s,early_stopping]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.56696, saving model to best_m.h5
 - 6s - loss: 0.6427 - acc: 0.6483 - val_loss: 0.5670 - val_acc: 0.6808
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.5251 - acc: 0.7389 - val_loss: 0.9002 - val_acc: 0.5087
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.4333 - acc: 0.7872 - val_loss: 0.9201 - val_acc: 0.5561
Epoch 4/100
Epoch 00004: val_loss improved from 0.56696 to 0.54017, saving model to best_m.h5
 - 3s - loss: 0.4619 - acc: 0.7645 - val_loss: 0.5402 - val_acc: 0.7456
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 3s - loss: 0.3970 - acc: 0.8020 - val_loss: 1.0398 - val_acc: 0.5486
Epoch 6/100
Epoch 00006: val_loss improved from 0.54017 to 0.36473, saving model to best_m.h5
 - 3s - loss: 0.4223 - acc: 0.8153 - val_loss: 0.3647 - val_acc: 0.8529
Epoch 7/100
Epoch 00007: val_loss improved from 0.36473 to 0.31066, saving model to best_m.h5
 - 3s - loss: 0.3760 - acc: 0.8183 - val_loss:

Epoch 64/100
Epoch 00064: val_loss did not improve
 - 3s - loss: 0.2547 - acc: 0.9006 - val_loss: 0.2324 - val_acc: 0.8953
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.2556 - acc: 0.8804 - val_loss: 0.2260 - val_acc: 0.9027
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.2372 - acc: 0.9000 - val_loss: 0.2106 - val_acc: 0.9177
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 3s - loss: 0.2464 - acc: 0.8914 - val_loss: 0.2195 - val_acc: 0.9027
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 3s - loss: 0.2318 - acc: 0.8956 - val_loss: 0.2252 - val_acc: 0.9002
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.54654, saving model to best_m.h5
 - 6s - loss: 0.6605 - acc: 0.6214 - val_loss: 0.5465 - val_acc: 0.7057
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.5331 - acc: 0.7339 - val_loss: 0.5941 - val_acc: 0.7007
Epoch 3/100
Epoch 00003: val_loss improved from 0.54654 to 0.49106, saving model to best_m.h5
 - 3s -

Epoch 58/100
Epoch 00058: val_loss did not improve
 - 3s - loss: 0.2363 - acc: 0.9014 - val_loss: 0.2376 - val_acc: 0.8903
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 3s - loss: 0.2200 - acc: 0.9081 - val_loss: 0.2360 - val_acc: 0.9052
Epoch 60/100
Epoch 00060: val_loss improved from 0.23274 to 0.23233, saving model to best_m.h5
 - 3s - loss: 0.2307 - acc: 0.8970 - val_loss: 0.2323 - val_acc: 0.9052
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 3s - loss: 0.2157 - acc: 0.9033 - val_loss: 0.2435 - val_acc: 0.9027
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 3s - loss: 0.2004 - acc: 0.9242 - val_loss: 0.2383 - val_acc: 0.8903
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 3s - loss: 0.2224 - acc: 0.9092 - val_loss: 0.2357 - val_acc: 0.8928
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 3s - loss: 0.2191 - acc: 0.9000 - val_loss: 0.2348 - val_acc: 0.9002
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 3s - loss: 0.2613 - acc: 0.8889 - val_los

Epoch 20/100
Epoch 00020: val_loss did not improve
 - 3s - loss: 0.2929 - acc: 0.8692 - val_loss: 0.2542 - val_acc: 0.9027
Epoch 21/100
Epoch 00021: val_loss improved from 0.24632 to 0.23516, saving model to best_m.h5
 - 3s - loss: 0.2942 - acc: 0.8731 - val_loss: 0.2352 - val_acc: 0.8953
Epoch 22/100
Epoch 00022: val_loss improved from 0.23516 to 0.23382, saving model to best_m.h5
 - 3s - loss: 0.2920 - acc: 0.8664 - val_loss: 0.2338 - val_acc: 0.8978
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 3s - loss: 0.2899 - acc: 0.8798 - val_loss: 0.2597 - val_acc: 0.8853
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 3s - loss: 0.2743 - acc: 0.8791 - val_loss: 0.2528 - val_acc: 0.9002
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 3s - loss: 0.2446 - acc: 0.8883 - val_loss: 0.2909 - val_acc: 0.8903
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 3s - loss: 0.3288 - acc: 0.8628 - val_loss: 0.2816 - val_acc: 0.8828
Epoch 27/100
Epoch 00027: val_loss did not improve


Epoch 21/100
Epoch 00021: val_loss did not improve
 - 3s - loss: 0.2830 - acc: 0.8714 - val_loss: 0.3730 - val_acc: 0.7955
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 3s - loss: 0.2652 - acc: 0.8892 - val_loss: 0.3013 - val_acc: 0.8504
Epoch 23/100
Epoch 00023: val_loss improved from 0.28982 to 0.27726, saving model to best_m.h5
 - 3s - loss: 0.2570 - acc: 0.8900 - val_loss: 0.2773 - val_acc: 0.8653
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 3s - loss: 0.2870 - acc: 0.8720 - val_loss: 0.2882 - val_acc: 0.8603
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 3s - loss: 0.2751 - acc: 0.8775 - val_loss: 0.2815 - val_acc: 0.8678
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 3s - loss: 0.2561 - acc: 0.8800 - val_loss: 0.2968 - val_acc: 0.8628
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 3s - loss: 0.2431 - acc: 0.9000 - val_loss: 0.3279 - val_acc: 0.8703
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 3s - loss: 0.2817 - acc: 0.8739 - val_los

In [10]:
import pickle
with open('../features/cnn_5_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_5_aug1_sub.csv', index=False)
# add zoom 0.231
# rm zoom rm flip 0.228
# only ud flip 0.230
# both ud lr flip 0.230

0.230234137733
         id  is_iceberg
0  5941774d    0.057277
1  4023181e    0.763656
2  b20200e4    0.023740
3  e7f018bb    0.995879
4  4371c8c3    0.466301
